In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import re
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
import textwrap
import progressbar
import keras
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1

seed_val = 2212
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# load all models and select roberta
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification


In [ ]:
#save the trained model

output_dir     = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Models/finetuned/legalbert_trained_on_single/'
pretrained_dir = None

input_file = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_train_dev.csv'
#test_file = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/c2_test.csv'
#pred_file = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/legalbert_trained_on_single_preprocessed_2_fixed_first510tokens_10eps_c2_test.csv'



#input_file = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_train_dev.csv'

#test_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_preprocessed_2.csv'
#pred_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_preprocessed_2_preds.csv'
#test_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_explanation_preprocessed_2.csv'
#pred_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_explanation_preprocessed_2_preds.csv'


#test_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_explanation.csv'
#pred_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_explanation_preds.csv'

#test_exp_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_explanation.csv'
#pred_exp_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_explanation_pred.csv'



#load data
df = pd.read_csv(input_file) # path to multi_dataset
df = df.applymap(lambda x: x.strip().replace('-\n', '').replace('\n', '') if isinstance(x, str) else x)
train_set = df.query(" split=='train' ")
#validation_set = df.query(" split=='dev' ")
validation_set = pd.read_csv(test_file)
#test_set = pd.read_csv(test_file)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")


if pretrained_dir == None:
  print('\n\nNo pretrained model')  
  model = BertForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", output_attentions=True)

else:
  model = BertForSequenceClassification.from_pretrained(pretrained_dir)
  print('\n\nLoad from ', pretrained_dir)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



Load from  /content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Models/finetuned/legalbert_trained_on_single_preprocessed_2_fixed_first510tokens_10eps/


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
validation_set['id'] = range(len(validation_set))
print(validation_set)

    Unnamed: 0                                   uid  t_ind  s_ind  \
0            0  c0e246e7-60f3-4523-8da7-056eb7887c1d      1      0   
1            1  47a5abfc-0258-4a1f-89bb-8ea70d8d466b      1      0   
2            2  16860b14-866f-4cb5-a82d-50bac0f0d2ff      1      0   
3            3  c455ccbf-3390-475f-b4e9-f72bce15a2cf      1      0   
4            4  b8cdf7a4-489a-460b-a511-16eae79b8912      1      0   
5            5  4ae07c84-2c72-455b-8154-587196becc0c      1      0   
6            6  2280d8b2-1223-44fc-bea0-577e2e1dfb4a      1      0   
7            7  b4035da9-0319-4ec1-b599-3ab987dc5b64      1      0   
8            8  14b24130-5ac7-4ba7-8fe0-cac06e59d6ab      1      0   
9            9  de4b65c9-ba8c-4f71-bd34-f6548db71cbe      1      0   
10          10  8ea3dc45-ac1f-4ddf-adfa-ed3fc6d425ab      1      0   
11          11  27d46dda-ba34-4fd6-bd27-f95861d49726      1      0   
12          12  40ce6f70-971f-4640-9fbc-00ce4dc948bf      1      0   
13          13  0287

In [ ]:
df

,Unnamed: 0,text,t_ind,s_ind,label,split,id
0,0,the manzil has a ground floor and three other ...,1,0,1,train,1
1,1,radha bai as her licensee. radha bai. it was s...,1,0,1,train,2
2,2,he retired in 1973. the rent has been paid sin...,1,0,0,train,3
3,3,the fact that the returning officer rejected t...,1,0,1,train,4
4,4,2 to 4.30 for rolling 1000 pieces. women and i...,1,0,0,train,5
...,...,...,...,...,...,...,...
5971,5971,the clinic was set up at bhilai somewhere in 1...,1,0,1,dev,5972
5972,5972,100. the state preferred appeals to the high c...,1,0,0,dev,5973
5973,5973,the licensee had numbervested right to a renew...,1,0,0,dev,5974
5974,5974,the plaintiff moved the high companyrt in revi...,1,0,1,dev,5975


# Preprocessing

# Preparing data

In [ ]:
def input_id_maker(dataf, tokenizer):
  input_ids = []
  lengths = []

  for i in progressbar.progressbar(range(len(dataf['text']))):
    sen = dataf['text'].iloc[i]
    sen = tokenizer.tokenize(sen)
    CLS = tokenizer.cls_token
    SEP = tokenizer.sep_token
    if(len(sen) > 510):
      sen = sen[:510]
    
    '''if len(sen) > 510:
      sen = sen[0:255] + sen[len(sen)-255:]'''

    sen = [CLS] + sen + [SEP]
    encoded_sent = tokenizer.convert_tokens_to_ids(sen)
    input_ids.append(encoded_sent)
    lengths.append(len(encoded_sent))

  input_ids = pad_sequences(input_ids, maxlen=512, value=0, dtype="long", truncating="post", padding="post")
  return input_ids, lengths

In [ ]:
train_input_ids, train_lengths = input_id_maker(train_set, tokenizer)
validation_input_ids, validation_lengths = input_id_maker(validation_set, tokenizer)

N/A% (0 of 4982) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--Token indices sequence length is longer than the specified maximum sequence length for this model (1800 > 512). Running this sequence through the model will result in indexing errors
100% (4982 of 4982) |####################| Elapsed Time: 0:01:57 Time:  0:01:57
100% (50 of 50) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [ ]:
def att_masking(input_ids):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks

In [ ]:
train_attention_masks = att_masking(train_input_ids)
validation_attention_masks = att_masking(validation_input_ids)

train_labels = train_set['label'].to_numpy().astype('int')
#validation_labels = validation_set['label'].to_numpy().astype('int')
validation_labels = validation_set['id'].to_numpy().astype('int')

In [ ]:
train_inputs = train_input_ids
validation_inputs = validation_input_ids
train_masks = train_attention_masks
validation_masks = validation_attention_masks

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
batch_size = 6
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)

# Training

In [ ]:
'''# max batch size should be 6 due to colab limits
batch_size = 6
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)

lr = 2e-6
max_grad_norm = 1.0
epochs = 10
num_total_steps = len(train_dataloader)*epochs
num_warmup_steps = 1000
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_total_steps)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    if (epoch_i+1)%1 == 0:
    # Create output directory if needed
      if not os.path.exists(output_dir):
          os.makedirs(output_dir)

      print("Saving model to %s (epoch %d)" % (output_dir, epoch_i+1))
      model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
      model_to_save.save_pretrained(output_dir)
      tokenizer.save_pretrained(output_dir)


    if (epoch_i+1)%5 == 0:
        
      print("")
      print("Running Validation...")

      t0 = time.time()

      model.eval()

      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      for batch in validation_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          
          with torch.no_grad():        
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      
          logits = outputs[0]

          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          eval_accuracy += tmp_eval_accuracy

          nb_eval_steps += 1

      # Report the final accuracy for this validation run.
      print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

print("")
print("Training complete!")


if not os.path.exists(output_dir):
  os.makedirs(output_dir)

print("Saving model to %s (epoch %d)" % (output_dir, epoch_i+1))
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)'''

'# max batch size should be 6 due to colab limits\nbatch_size = 6\ntrain_data = TensorDataset(train_inputs, train_masks, train_labels)\ntrain_sampler = RandomSampler(train_data)\ntrain_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)\n\nlr = 2e-6\nmax_grad_norm = 1.0\nepochs = 10\nnum_total_steps = len(train_dataloader)*epochs\nnum_warmup_steps = 1000\nwarmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1\noptimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)\nscheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_total_steps)\n\nloss_values = []\n\n# For each epoch...\nfor epoch_i in range(0, epochs):\n    print(\'======== Epoch {:} / {:} ========\'.format(epoch_i + 1, epochs))\n    print(\'Training...\')\n\n    t0 = time.time()\n    total_loss = 0\n\n    model.train()\n\n    for step, batch in enumerate(train_dataloader):\n        if step % 40 == 0 and not 

#Predicting

## Evaluation on training set

**last 510 tokens**

**preprocessed_2**

10 epochs
Predicting labels for 4,982 test sentences...
0.9411882778000803
0.9515030558839894 0.9257865202796357 0.9384686457828602 0.9411882778000803 0.9411882778000803 0.9411882778000803


20 epochs
Predicting labels for 4,982 test sentences...
0.9937775993576877
0.9940610263028817 0.9927478603638804 0.9934040093699549 0.9937775993576877 0.9937775993576877 0.9937775993576877


**preprocessed_2_fixed**

10epochs
Predicting labels for 4,982 test sentences...
0.735046166198314
0.7959920222724617 0.6613469335398029 0.7224494767706652 0.735046166198314 0.735046166198314 0.7350461661983139

20 epochs
Predicting labels for 4,982 test sentences...
0.8898032918506624
0.9201835102106448 0.8570681053944263 0.8875051014573739 0.8898032918506624 0.8898032918506624 0.8898032918506624

40 epochs
Predicting labels for 4,982 test sentences...
0.9857486953030912
0.9887444514901712 0.9813059505002633 0.9850111578781171 0.9857486953030912 0.9857486953030912 0.9857486953030912

**FIRST 510 TOKENS**

**preprocessed_2_fixed**

Predicting labels for 4,982 test sentences...
0.8613006824568447
0.8871609770357618 0.8257470129984592 0.8553530383397178 0.8613006824568447 0.8613006824568447 0.8613006824568447

In [ ]:
'''batch_size = 6

prediction_data = TensorDataset(train_inputs, train_masks, train_labels)
prediction_sampler = SequentialSampler(train_data)
prediction_dataloader = DataLoader(train_data, shuffle=False, sampler=None, batch_size = batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)

      logits = outputs.logits
      
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

print(flat_accuracy(predictions,true_labels))

macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)

print('    DONE.')'''

"batch_size = 6\n\nprediction_data = TensorDataset(train_inputs, train_masks, train_labels)\nprediction_sampler = SequentialSampler(train_data)\nprediction_dataloader = DataLoader(train_data, shuffle=False, sampler=None, batch_size = batch_size)\n\nprint('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))\nmodel.eval()\n\npredictions , true_labels = [], []\nexplanations = []\n\nfor (step, batch) in enumerate(prediction_dataloader):\n  batch = tuple(t.to(device) for t in batch)\n  \n  b_input_ids, b_input_mask, b_labels = batch\n  #print(b_input_ids)\n\n  with torch.no_grad():\n      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)\n\n      logits = outputs.logits\n      attentions = outputs.attentions\n      \n  # Move logits and labels to CPU\n  logits = logits.detach().cpu().numpy()\n  label_ids = b_labels.to('cpu').numpy()\n  attentions = attentions[0].detach().cpu().numpy()\n  \n  # Store predictions and

##evaluation on dev set

**last 510 tokens**

**preprocessed_2**

10 epochs
Predicting labels for 994 test sentences...
0.5985915492957746
0.7012910798122065 0.5985915492957746 0.6458843353575175 0.5985915492957746 0.5985915492957746 0.5985915492957746

20 epcohs
Predicting labels for 994 test sentences...
0.6297786720321932
0.6888731771984091 0.6297786720321932 0.6580017841521162 0.6297786720321932 0.6297786720321932 0.6297786720321932


**preprocessed_2_fixed**

10 epochs
Predicting labels for 994 test sentences...
0.5382293762575453
0.5829905786402306 0.5382293762575453 0.5597165018957676 0.5382293762575453 0.5382293762575453 0.5382293762575453

20 epochs
Predicting labels for 994 test sentences...
0.5553319919517102
0.5920909893338184 0.5553319919517102 0.5731226825438435 0.5553319919517102 0.5553319919517102 0.5553319919517102

40 epcohs 
Predicting labels for 994 test sentences...
0.545271629778672
0.5686426164300316 0.545271629778672 0.5567119502739849 0.545271629778672 0.545271629778672 0.545271629778672


**FIRST 510 TOKENS**

**preprocessed_2_fixed**

10 epochs
0.6287726358148893
0.690558351306015 0.6287726358148893 0.6582187471882317 0.6287726358148893 0.6287726358148893 0.6287726358148893
  

In [ ]:
'''batch_size = 1

# Create the DataLoader.
prediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, shuffle=False, sampler=None, batch_size = batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)
      logits = outputs.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

print(flat_accuracy(predictions,true_labels))

macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)

print('    DONE.')'''

"batch_size = 1\n\n# Create the DataLoader.\nprediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)\nprediction_sampler = SequentialSampler(prediction_data)\nprediction_dataloader = DataLoader(prediction_data, shuffle=False, sampler=None, batch_size = batch_size)\n\nprint('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))\nmodel.eval()\n\npredictions , true_labels = [], []\nexplanations = []\n\nfor (step, batch) in enumerate(prediction_dataloader):\n  batch = tuple(t.to(device) for t in batch)\n  \n  b_input_ids, b_input_mask, b_labels = batch\n  #print(b_input_ids)\n\n  with torch.no_grad():\n      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)\n      logits = outputs.logits\n      attentions = outputs.attentions[0]\n\n  # Move logits and labels to CPU\n  logits = logits.detach().cpu().numpy()\n  label_ids = b_labels.to('cpu').numpy()\n  attentions = attentions.detach().c

## Prediction on binary classification test set

In [ ]:
"""batch_size = 1

# Create the DataLoader.
prediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, shuffle=False, sampler=None, batch_size = batch_size)
assert len(prediction_data) == 1500

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)

      logits = outputs.logits
      
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()


print('    DONE.')

validation_set['id2'] = labels_flat
validation_set['prediction'] = pred_flat
assert all(validation_set['id2'] == validation_set['id'])
validation_set.to_csv(pred_file, index=False,)"""

"batch_size = 1\n\n# Create the DataLoader.\nprediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)\nprediction_sampler = SequentialSampler(prediction_data)\nprediction_dataloader = DataLoader(prediction_data, shuffle=False, sampler=None, batch_size = batch_size)\nassert len(prediction_data) == 1500\n\nprint('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))\nmodel.eval()\n\npredictions , true_labels = [], []\nexplanations = []\n\nfor (step, batch) in enumerate(prediction_dataloader):\n  batch = tuple(t.to(device) for t in batch)\n  \n  b_input_ids, b_input_mask, b_labels = batch\n  #print(b_input_ids)\n\n  with torch.no_grad():\n      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)\n\n      logits = outputs.logits\n      attentions = outputs.attentions[0]\n      \n  # Move logits and labels to CPU\n  logits = logits.detach().cpu().numpy()\n  label_ids = b_labels.to('cpu').

##Prediction on explanation test set

In [ ]:
batch_size = 1
window_size = 128

# Create the DataLoader.
prediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, shuffle=False, sampler=None, batch_size = batch_size)
assert len(prediction_data) == 50

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, output_attentions=True)

      logits = outputs.logits
      attentions = outputs.attentions[-1]

      #max_ind = attentions[0][-1][0].detach().cpu().numpy().argmax()

      max_inds = np.argpartition(attentions[0][-1][0].detach().cpu().numpy(), -3)[-3:]
     
      explanation_sents = []
      for max_ind in max_inds:
        selected_tokens = b_input_ids[0][max_ind-int(window_size/2):max_ind+int(window_size/2)]
        tokens = tokenizer.convert_ids_to_tokens(selected_tokens) 
        #print(tokens)
        sent = tokenizer.convert_tokens_to_string(tokens)
        #print(sent)
        explanation_sents.append(sent)

      explanation_sents = " ".join(explanation_sents)
      explanations.append(explanation_sents) 

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  attentions = attentions.detach().cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

print('    DONE.')


decisions = ['Accepted' if _ == 1 else 'Denied' for _ in pred_flat]

validation_set['id2'] = labels_flat
validation_set['prediction'] = pred_flat
validation_set['decision'] = decisions
validation_set['explanation'] = explanations
assert all(validation_set['id2'] == validation_set['id'])
validation_set.to_csv(pred_file, index=False, columns=['uid','decision','explanation'] )

Predicting labels for 50 test sentences...
    DONE.
